In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import date, datetime

pd.options.plotting.backend = "plotly"

In [ ]:
df = pd.read_csv("heart_rate.csv", parse_dates=['timestamp'])
df['local_time'] = df['timestamp'].dt.tz_localize('utc').dt.tz_convert('Europe/Paris')
df

In [ ]:
df.plot(x='timestamp', y='heart_rate', kind='line', title='Heart rate', width=1000, height=500)

In [ ]:
sleep_end = "07:30" #Sleep start time
sleep_start = "23:00" #Sleep end time


#Return the new waking HR list
awake_df = df.set_index('local_time').between_time(sleep_end, sleep_start).reset_index()
awake_df.plot(x='local_time', y='heart_rate', kind='line', title='Heart rate', width=1000, height=500)

In [ ]:
max_hr = 190 #Max heart rate

#Create bins based on max HR
rest = max_hr * 0.4
stand = max_hr * 0.5
z0 = max_hr * 0.6
z1 = max_hr * 0.7
z2 = max_hr * 0.8
z3_4 = max_hr * 0.9
z5 = max_hr

heart_rate_list = awake_df['heart_rate'].tolist()
days = (max(awake_df['local_time']) - min(awake_df['local_time'])).days

#Set the distribution bins that I want to look at for zones (yours might be different)
hist, bins = np.histogram(heart_rate_list, bins=[0, rest, stand, z0, z1, z2, z3_4, z5])
hist = (hist * 2 / 60 / days).tolist()  #2 min samples over days
bin_names = ['Sit/Relax: <40% MHR', 'Stand/Move: 40-50% MHR', 'Z0 Exercise: 50-60% MHR', 'Z1 Exercise: 60-70% MHR', 'Z2 Exercise: 70-80% MHR', 'Z3/4 Exercise: 80-90% MHR', 'Z5+: >90% MHR']

print(hist)

#Build the chart
fig = go.Figure()
fig.add_trace(go.Bar(x=bin_names, y=hist, marker=dict(color=['red','blue','blue','blue','blue','blue','blue'])))
fig.update_layout(title_text=f'Daily Waking Heart Rate Zone Distribution', yaxis_title='Hours')
fig.show()